In [2]:
!pip install PyPDF2

     |████████████████████████████████| 77 kB 8.3 MB/s eta 0:00:011
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61084 sha256=5acdbe29ef215b3ab75d7353ae1245b23eb873e093944cfac86706ca229544f4
  Stored in directory: /Users/sdeshpande/Library/Caches/pip/wheels/b1/1a/8f/a4c34be976825a2f7948d0fa40907598d69834f8ab5889de11
Successfully built PyPDF2


In [3]:
import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

In [4]:
with open('/Users/sdeshpande/Desktop/text_analysis_scripts/sample_text.pdf','rb') as pdf_file, open('/Users/sdeshpande/Desktop/text_analysis_scripts/sample.txt', 'w') as text_file:
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    for page_number in range(number_of_pages):   
        page = read_pdf.getPage(page_number)
        page_content = page.extractText()
        text_file.write(page_content)

In [5]:
with open('/Users/sdeshpande/Desktop/text_analysis_scripts/sample.txt', 'r', encoding="latin-1") as f:
    read_output = f.readlines()
read_output = '\n'.join(read_output).strip().replace('\n','')

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;.]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
SHORTWORD = re.compile(r'\W*\b\w{1,3}\b')

stop_words_lst = ['cell','spectral','data','double','bond','barrier','soluble', 'cells', 'performed using']

In [7]:
def clean_text(text):
    """
        text: a string
        return: modified initial string
    """
    text = str(text).lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    text = re.sub(r'\b[a-zA-Z]\b','',str(text)) # remove single letter words
    text = re.sub('\d+','',text) # remove digits
    text = SHORTWORD.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    for w in stop_words_lst:
        pattern = r'\b'+w+r'\b'
        text = re.sub(pattern, '', str(text))

    return text

In [8]:
pdf_o = clean_text(read_output)
text_tokens = word_tokenize(pdf_o)

pdf_i=[pdf_o]

In [9]:
# Getting bigrams
vectorizer = CountVectorizer(ngram_range =(2, 3))
X1 = vectorizer.fit_transform(pdf_i)
features = (vectorizer.get_feature_names())

X2 = vectorizer.fit_transform(pdf_i)
scores = (X2.toarray())

sums = X2.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(50))



Words : 
                                   term  rank
5583                  pluripotent stem    20
1365                 cluster formation    11
2792                   fisher scientic    10
2506              ermo fisher scientic    10
2505                       ermo fisher    10
1154             cellcluster formation     7
4626                    medium changed     7
5090                       normal hdfs     7
2550                        every days     6
290                        alexa fluor     6
4629                 medium containing     6
3928          induced pluripotent stem     5
3927               induced pluripotent     5
4628              medium changed fresh     5
6396                     rics cultured     5
4965                         natl acad     5
6325                ribosomes isolated     5
2812                  fluor phalloidin     5
4635                      medium every     5
6263            ribosome incorporation     5
98                  acidophilus lysate     